In [42]:
# Load relevant packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as sm
import warnings
from sklearn.cluster import KMeans 

In [43]:
## load data set
carmax_df = pd.read_csv('CaseDataRemodeled.csv')
top10_df = pd.read_csv('Top10ByIncome.csv')

In [44]:
cluster_df = carmax_df
cluster_df = cluster_df.drop('insert_num', axis=1)

## Time to Cluster - Gonna use k-means initially 

In general, we can cluster observations on the basis of the features in order to identify subgroups among the observations, or we can cluster fea- tures on the basis of the observations in order to discover subgroups among the features. In what follows, for simplicity we will discuss clustering obser- vations on the basis of the features, though the converse can be performed by simply transposing the data matrix.

In [45]:
cluster_df = carmax_df
cluster_df = cluster_df.drop('insert_num', axis=1)
cluster_df = cluster_df.drop('customer_age', axis=1)
cluster_df = cluster_df.drop('post_purchase_satisfaction', axis=1)
cluster_df.head()

,purchase_make,purchase_model,purchase_vehicle_year,purchase_price_LB,purchase_price_UB,trade_in,vehicle_financing,customer_income,customer_gender,customer_previous_purchase,customer_distance_to_dealer,vehicle_warranty_used,subsequent_purchases,AgeBin,purchase_class
0,DODGE,CHARGER,2012,15001.0,20000.0,1,1,40001 - 60000,U,0,8.0,0,1,Twenties,sedan
1,FORD,F150,2007,15001.0,20000.0,0,0,0 - 20000,F,1,19.0,0,0,Fifties,truck
2,BMW,328,2010,25001.0,30000.0,1,1,60001 - 80000,F,1,21.0,0,0,Forties,sedan
3,LEXUS,GS 300,2003,10001.0,15000.0,0,1,20001 - 40000,M,0,8.0,0,0,Twenties,sedan
4,CHEVROLET,CRUZE,2012,15001.0,20000.0,1,1,120001 - 140000,M,1,5.0,1,1,Thirties,sedan


In [46]:
# function to transform into numerical bins by normal/gaussian distribution
def categorical_transform(col):
    classnames, factorval, rank = np.unique(col, return_inverse = True,return_counts = True)
    
    temp = np.array(rank)
    temp.sort()
    temp = np.concatenate((temp[::2], temp[~np.isin(temp,temp[::2])][::-1]))

    category_count = {}
    for i in range(len(classnames)):
        category_count[classnames[i]] = rank[i]
    
    vals = [i * 5 for i in range(len(temp))]
    
    for i in range(len(temp)):
        for key, value in category_count.items():
            if value == temp[i]:
                category_count[key] = vals[i]
            
    check_list = list(col)
    for i in range(len(check_list)):
        for key, value in category_count.items():
            if check_list[i] == key:
                factorval[i] = value 
                break
    
    return category_count, factorval

In [47]:
## references of transformation for later 
references = {}

for i in cluster_df.columns:
    if cluster_df[i].dtype == 'object':
        classes = cluster_df[i]
        references[i], cluster_df[i] = categorical_transform(classes)